In [48]:
import pandas as pd
import pypfopt
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

from pypfopt import HRPOpt
from pypfopt.efficient_frontier import EfficientCVaR

In [49]:
portfolio = pd.read_csv("data/Stock_History/closing.csv")
company = pd.read_csv("data/Company_data.csv")

In [50]:
portfolio = portfolio.drop(columns=['GEHC', 'CEG', 'OGN', 'CARR', 'OTIS', 'WTW'])

In [51]:
company = company.loc[company['Symbol'].isin(portfolio.columns)]

In [80]:
def Mean_Variance_Optimization(data,latest_prices,portfolio_value):
    
    Mean_Historical_Return = mean_historical_return(data)
    Covariance_Shrinkage = CovarianceShrinkage(data).ledoit_wolf()

    Efficient_Frontier = EfficientFrontier(Mean_Historical_Return, Covariance_Shrinkage)
    weights = Efficient_Frontier.max_sharpe()
    
    cleaned_weights = Efficient_Frontier.clean_weights()
    
    performance = Efficient_Frontier.portfolio_performance(verbose=False)

    Discrete_Allocation = DiscreteAllocation(weights, latest_prices, total_portfolio_value=portfolio_value)
    
    allocation, leftover = Discrete_Allocation.greedy_portfolio()
    
    allocation={key: value for key, value in allocation.items() if value >= 0}
    
    latest_prices.loc[list(allocation.keys())]

    value=latest_prices.loc[list(allocation.keys())]*list(allocation.values())

    value[value > 0].sum()

    performance = list(performance)

    performance[0] = performance[0]*100
    performance[1] = performance[1]*100


    return(allocation,leftover,performance)

In [79]:
def Hierarchical_Risk_parity(data,latest_prices,portfolio_value):
    
    returns = data.pct_change().dropna()

    hrp = HRPOpt(returns)
    hrp_weights = hrp.optimize()

    hrp.portfolio_performance(verbose=False)
    
    print(dict(hrp_weights))

    da_hrp = DiscreteAllocation(hrp_weights, latest_prices, total_portfolio_value=100000)
    
    allocation, leftover = da_hrp.greedy_portfolio() 
    
    print("Discrete allocation (HRP):", allocation)
    
    print("Funds remaining (HRP): ${:.2f}".format(leftover))


In [54]:
def CVAR(data,latest_prices,portfolio_value):
    S = data.cov()
    mu = mean_historical_return(data)

    ef_cvar = EfficientCVaR(mu, S)
    cvar_weights = ef_cvar.min_cvar()

    ef_cvar.portfolio_performance(verbose=True)
    
    cleaned_weights = ef_cvar.clean_weights()
    print(dict(cleaned_weights))

    da_cvar = DiscreteAllocation(cvar_weights, latest_prices, total_portfolio_value=100000)
    
    allocation, leftover = da_cvar.greedy_portfolio()
    print("Discrete allocation (CVAR):", allocation)
    print("Funds remaining (CVAR): ${:.2f}".format(leftover))
    

In [55]:
total_portfolio_value=100000
latest_prices = get_latest_prices(portfolio.iloc[:, 1:])

In [81]:
Mean_Variance_Optimization(portfolio.iloc[:, 1:],latest_prices,total_portfolio_value)

f:\anaconda\envs\Project\lib\site-packages\cvxpy\problems\problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
f:\anaconda\envs\Project\lib\site-packages\pypfopt\discrete_allocation.py:212: RuntimeWarning: divide by zero encountered in divide
  current_weights /= current_weights.sum()
f:\anaconda\envs\Project\lib\site-packages\pypfopt\discrete_allocation.py:212: RuntimeWarning: invalid value encountered in divide
  current_weights /= current_weights.sum()


({'MCK': 55,
  'LLY': 56,
  'MRNA': 100,
  'PWR': 108,
  'ENPH': 50,
  'EQT': 175,
  'TSCO': 13,
  'KR': 46,
  'MOH': 3,
  'TSLA': 3},
 26.231608127956633,
 [63.8467533675485, 30.063917324158968, 2.0571754738644543])

In [11]:
Hierarchical_Risk_parity(portfolio.iloc[:, 1:],latest_prices,total_portfolio_value)

Expected annual return: 12.3%
Annual volatility: 23.0%
Sharpe Ratio: 0.45
{'A': 0.003674993888510793, 'AAL': 0.0004515025862375373, 'AAP': 0.0013331062279617488, 'AAPL': 0.0016536041449563984, 'ABBV': 0.0053642181736963, 'ABC': 0.0022686558169749574, 'ABT': 0.0028454246016717324, 'ACGL': 0.0012272099759233126, 'ACN': 0.0019050602213808705, 'ADBE': 0.0013023148455898657, 'ADI': 0.0011317429358723469, 'ADM': 0.0030938932563764044, 'ADP': 0.0019038586862115147, 'ADSK': 0.0013079253635319447, 'AEE': 0.0021543229201328017, 'AEP': 0.002497445369041483, 'AES': 0.001480903275373121, 'AFL': 0.0021004079726282474, 'AIG': 0.0007154327890289945, 'AIZ': 0.0022782828677433485, 'AJG': 0.0023897972485647616, 'AKAM': 0.004676489571263145, 'ALB': 0.001380321661865856, 'ALGN': 0.00104365838230894, 'ALK': 0.0007794624517292614, 'ALL': 0.0022510362007932914, 'ALLE': 0.0016495481269588543, 'AMAT': 0.0008302021352676702, 'AMCR': 0.0022179023547927164, 'AMD': 0.0009373366840687358, 'AME': 0.001619784652934938

In [12]:
CVAR(portfolio.iloc[:, 1:],latest_prices,total_portfolio_value)

Expected annual return: 1.4%
Conditional Value at Risk: -1794.29%
{'A': 0.0, 'AAL': 0.0, 'AAP': 0.0, 'AAPL': 0.0, 'ABBV': 0.0, 'ABC': 0.0, 'ABT': 0.0, 'ACGL': 0.0, 'ACN': 0.0, 'ADBE': 0.0, 'ADI': 0.0, 'ADM': 0.0, 'ADP': 0.0, 'ADSK': 0.0, 'AEE': 0.0, 'AEP': 0.0, 'AES': 0.0, 'AFL': 0.0, 'AIG': 0.0, 'AIZ': 0.0, 'AJG': 0.0, 'AKAM': 0.0, 'ALB': 0.0, 'ALGN': 0.0, 'ALK': 0.0, 'ALL': 0.0, 'ALLE': 0.0, 'AMAT': 0.0, 'AMCR': 0.0, 'AMD': 0.0, 'AME': 0.0, 'AMGN': 0.0, 'AMP': 0.0, 'AMT': 0.0, 'AMZN': 0.0, 'ANET': 0.0, 'ANSS': 0.0, 'AON': 0.0, 'AOS': 0.0, 'APA': 0.0, 'APD': 0.0, 'APH': 0.0, 'APTV': 0.0, 'ARE': 0.0, 'ATO': 0.0, 'ATVI': 0.0, 'AVB': 0.0, 'AVGO': 0.0, 'AVY': 0.0, 'AWK': 0.0, 'AXON': 0.0, 'AXP': 0.0, 'AZO': 0.0, 'BA': 0.0, 'BAC': 0.0, 'BALL': 0.0, 'BAX': 0.0, 'BBWI': 0.0, 'BBY': 0.0, 'BDX': 0.0, 'BEN': 0.0, 'BF-B': 0.0, 'BG': 0.0, 'BIIB': 0.0, 'BIO': 0.0, 'BK': 0.0, 'BKNG': 0.02856, 'BKR': 0.0, 'BLK': 0.0, 'BMY': 0.0, 'BR': 0.0, 'BRK-B': 0.0, 'BRO': 0.0, 'BSX': 0.0, 'BWA': 0.0, 'BXP': 0.0

f:\anaconda\envs\project\lib\site-packages\pypfopt\discrete_allocation.py:212: RuntimeWarning: invalid value encountered in divide
  current_weights /= current_weights.sum()


In [20]:
divers= {'Communication Services': 6.0,
 'Consumer Discretionary': 9.0,
 'Consumer Staples': 8.0,
 'Energy': 15.0,
 'Financials': 4.0,
 'Industrials': 9.0,
 'Materials': 11.0,
 'Real Estate': 2.0,
 'Information Technology': 23.0,
 'Utilities': 4.0,
 'Health Care': 9.0}

In [13]:
for key, value in divers.items():
    divers[key] = total_portfolio_value/100*value

In [88]:
allocation = []
leftover = 0
performance = []

for key, value in divers.items():
    allo,leftover,perf = Mean_Variance_Optimization(portfolio[list(company[company['GICS Sector'] == key].Symbol)],
                               get_latest_prices(portfolio[list(company[company['GICS Sector'] == key].Symbol)]),
                               (value+leftover))
    allocation.extend(list(allo.items()))
    performance.extend([perf]) 
    

f:\anaconda\envs\Project\lib\site-packages\pypfopt\discrete_allocation.py:212: RuntimeWarning: invalid value encountered in divide
  current_weights /= current_weights.sum()
f:\anaconda\envs\Project\lib\site-packages\pypfopt\discrete_allocation.py:212: RuntimeWarning: divide by zero encountered in divide
  current_weights /= current_weights.sum()
f:\anaconda\envs\Project\lib\site-packages\pypfopt\discrete_allocation.py:212: RuntimeWarning: invalid value encountered in divide
  current_weights /= current_weights.sum()
f:\anaconda\envs\Project\lib\site-packages\pypfopt\discrete_allocation.py:212: RuntimeWarning: divide by zero encountered in divide
  current_weights /= current_weights.sum()
f:\anaconda\envs\Project\lib\site-packages\pypfopt\discrete_allocation.py:212: RuntimeWarning: invalid value encountered in divide
  current_weights /= current_weights.sum()
f:\anaconda\envs\Project\lib\site-packages\pypfopt\discrete_allocation.py:212: RuntimeWarning: divide by zero encountered in div

{}
{}
{}
{'EQT': 1}


f:\anaconda\envs\Project\lib\site-packages\pypfopt\discrete_allocation.py:212: RuntimeWarning: divide by zero encountered in divide
  current_weights /= current_weights.sum()
f:\anaconda\envs\Project\lib\site-packages\pypfopt\discrete_allocation.py:212: RuntimeWarning: invalid value encountered in divide
  current_weights /= current_weights.sum()
f:\anaconda\envs\Project\lib\site-packages\pypfopt\discrete_allocation.py:212: RuntimeWarning: invalid value encountered in divide
  current_weights /= current_weights.sum()
f:\anaconda\envs\Project\lib\site-packages\pypfopt\discrete_allocation.py:212: RuntimeWarning: divide by zero encountered in divide
  current_weights /= current_weights.sum()
f:\anaconda\envs\Project\lib\site-packages\pypfopt\discrete_allocation.py:212: RuntimeWarning: invalid value encountered in divide
  current_weights /= current_weights.sum()
f:\anaconda\envs\Project\lib\site-packages\pypfopt\discrete_allocation.py:212: RuntimeWarning: divide by zero encountered in div

{}
{'AAL': 1}
{'AMCR': 1}
{}
{}
{'PCG': 1}
{}


f:\anaconda\envs\Project\lib\site-packages\pypfopt\discrete_allocation.py:212: RuntimeWarning: divide by zero encountered in divide
  current_weights /= current_weights.sum()
f:\anaconda\envs\Project\lib\site-packages\pypfopt\discrete_allocation.py:212: RuntimeWarning: invalid value encountered in divide
  current_weights /= current_weights.sum()
f:\anaconda\envs\Project\lib\site-packages\pypfopt\discrete_allocation.py:212: RuntimeWarning: divide by zero encountered in divide
  current_weights /= current_weights.sum()
f:\anaconda\envs\Project\lib\site-packages\pypfopt\discrete_allocation.py:212: RuntimeWarning: invalid value encountered in divide
  current_weights /= current_weights.sum()


In [86]:
result = [sum(elements) for elements in zip(*performance)]
result = [value / len(performance) for value in result]

print(result)



[35.971034650526796, 35.7793718284415, 0.9345763771568737]


In [87]:
allocation

[('EQT', 1), ('AAL', 1), ('AMCR', 1), ('PCG', 1)]